In [ ]:
import os
import pandas as pd
import zipfile

In [ ]:
def read_csvs_from_zip(zip_path: str, csv_path: str, *args, **kwargs) -> pd.DataFrame:
    with zipfile.ZipFile(zip_path) as zip:
        with zip.open(csv_path) as csv:
            data = pd.read_csv(csv, *args, **kwargs)
    return data

Load `children_in_poverty` data

In [ ]:
children_in_poverty = pd.read_excel(
    '../../data/raw/neet-factors/children-in-low-income-families-local-area-statistics-2014-to-2022.ods',
    sheet_name='4_Absolute_Local_Authority',
    skiprows=9,
    usecols=[1, 17],
    names=['geography_code', 'Children in poverty'],
    index_col=0
).iloc[:, 0].mul(100).round(1)

Load `children_looked_after` data

In [ ]:
children_looked_after = read_csvs_from_zip(
          '../../data/raw/neet-factors/cla-all.zip',
          'data/cla_number_and_rate_per_10k_children.csv',
          usecols=[
            'time_period',
            'geographic_level',
            'population_count',
            'new_la_code',
            'rate_per_10000'
          ],
          index_col='new_la_code'
        )

children_looked_after = children_looked_after.loc[
  (children_looked_after.geographic_level == 'Local authority') &
  (children_looked_after.population_count == 'Children looked after at 31 March each year') &
  (children_looked_after.time_period == 2022),
  ["rate_per_10000"]
].rename(columns={
  'rate_per_10000': 'Children looked after'
})

Load `health_disability` data

In [ ]:
health_disability_2021 = pd.read_csv('../../data/raw/neet-factors/health_disability_2021.csv', index_col='Lower tier local authorities Code')

sum = health_disability_2021.loc[
  (health_disability_2021['Disability (3 categories)'] == 'Disabled under the Equality Act') &
  (health_disability_2021['Age (C) (4 categories)'].isin(['Aged 15 years and under', 'Aged 16 to 24 years' ])),
  "Observation"
].groupby('Lower tier local authorities Code').sum()

total = health_disability_2021.loc[
  (health_disability_2021['Age (C) (4 categories)'].isin(['Aged 15 years and under', 'Aged 16 to 24 years' ])),
  "Observation"
].groupby('Lower tier local authorities Code').sum()

health_disability_2021 = (sum/total).mul(100).to_frame('Disability (age < 25)')

Load `family_disability_2021` data

In [109]:
family_disability_2021 = pd.read_csv(
  filepath_or_buffer='../../data/raw/neet-factors/family_disability_2021.csv',
  index_col='Lower tier local authorities Code'
)

sum = family_disability_2021.loc[
  family_disability_2021['Disability - Equality act disabled (4 categories) Code'].isin([1]),
  "Observation"
].groupby('Lower tier local authorities Code').sum()

total = family_disability_2021.loc[
  :,
  "Observation"
].groupby('Lower tier local authorities Code').sum()

family_disability_2021 = (sum/total).mul(100).to_frame('Disability (all)')

Load `economic_inactivity` data.

This is not the same value as in the spreadsheet. Need to double check logic.

In [113]:
economic_inactivity = pd.read_csv(
  '../../data/raw/neet-factors/economic_inactivity_status.csv',
  index_col='Lower tier local authorities Code'
)

in_age_range = economic_inactivity['Age (C) (4 categories)'] == 'Aged 16 to 24 years'
is_economically_inactive = economic_inactivity['Economic activity status (7 categories)'] == 'Economically inactive (excluding full-time students)'

val = economic_inactivity.loc[in_age_range & is_economically_inactive, 'Observation'].groupby('Lower tier local authorities Code').sum()
total = economic_inactivity.loc[in_age_range, 'Observation'].groupby('Lower tier local authorities Code').sum()

economic_inactivity = ( val / total ).mul(100).to_frame('Economic inactivity (NEET)')

Create base data frame

In [114]:
local_authorities = pd.read_csv('../../data/reference/local_authorities.csv', index_col=[0])

Collate all layers into a single file

In [115]:
data = (
  local_authorities
    .join(children_in_poverty)
    .join(children_looked_after)
    .join(health_disability_2021)
    .join(family_disability_2021)
    # .join(economic_inactivity)
    .set_index(['Local Authority Name', 'Group'], append=True)
  )

Save to a CSV file

In [ ]:
SOURCES_CSV='../../data/processed/yff/neet-factors-sources.csv'
os.makedirs(os.path.dirname(SOURCES_CSV), exist_ok=True)
data.melt(ignore_index=False).to_csv(SOURCES_CSV)

In [ ]:
data

In [ ]:
children_looked_after


In [ ]:
data.columns